# Colab Notebook for Cellpose segmentation

Author: Dr.Yusuke Kimata (Ueda G)<br>
ref: GitHub (https://github.com/MouseLand/cellpose)<br>
<br>
Input File Requirements : tiff stack, located in `/content/drive/MyDrive/Colab Notebooks/Cellpose/input`


In [ ]:
# ============================================================
# 0) Environment setup (run once per Colab runtime) ───────────
# ============================================================
!pip install -q cellpose tifffile ipywidgets scikit-image opencv-python-headless

import torch, os, subprocess
print("PyTorch detects GPU:", torch.cuda.is_available())
!nvidia-smi  # GPU model (optional)

In [ ]:
# ============================================================
# 1) Load a 3-D / 4-D TIFF stack  ─────────────────────────────
#    Expected axis order: (Z, Y, X) or (Z, C, Y, X)
# ============================================================
import tifffile as tif
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

import os
HOME = os.getcwd()
print("HOME:", HOME)

%cd {HOME}/Colab Notebooks/Cellpose


In [ ]:
img_name = '1_original' # <-- change to your own file
tiff_path = './input/' + img_name + '.tif'
stack = tif.imread(tiff_path)
print("Input shape:", stack.shape)

import ipywidgets as widgets

z_mid  = stack.shape[0] // 2

def show_raw(z):
    """Interactive viewer for raw slices."""
    plt.figure(figsize=(4, 4))
    plt.imshow(stack[z], cmap='gray')
    plt.title(f'Raw slice  z={z}')
    plt.axis('off')
    plt.show()

widgets.interact(
    show_raw,
    z=widgets.IntSlider(min=0, max=stack.shape[0]-1,
                        step=1, value=z_mid,
                        description='Z-slice',
                        continuous_update=False)
);

In [ ]:
# ============================================================
# 2) Axis semantics & Cellpose model  ────────────────────────
# ============================================================
z_axis       = 0          # index of Z
channel_axis = None       # None if single-channel, else e.g. 1 for (Z,C,Y,X)

from cellpose import models

model = models.CellposeModel(gpu=True)
print("Model path:", model.pretrained_model)

In [ ]:
# ============================================================
# 3) Run 3-D Cellpose segmentation  ──────────────────────────
# ============================================================
masks, flows, styles = model.eval(
        stack,
        do_3D=True,
        z_axis=z_axis,
        channel_axis=channel_axis,
        channels=[0, 0],       # [cytoplasm, nucleus] or [chan, chan2]
        diameter=30,           # pixel diameter (mandatory in 3-D)
        flow_threshold=0.4,
        min_size=100,
        batch_size=2,
)
print("Segmentation finished ➜ labels:", masks.max())

In [ ]:
# ============================================================
# 4) Overlay slider (raw + mask) ─────────────────────────────
# ============================================================

from matplotlib.colors import ListedColormap, BoundaryNorm, hsv_to_rgb

def large_discrete_cmap(n_labels, seed=0):
    """
    Return a ListedColormap with `n_labels` visibly distinct colors.
    Color i depends only on i (and seed), thus the color assignment is deterministic.
    """
    phi = (1 + 5 ** 0.5) / 2                 # Golden ratio
    hues = (np.arange(n_labels) / phi) % 1.0 # Uniform distribution in [0, 1)
    rng  = np.random.default_rng(seed)
    sats = rng.uniform(0.6, 0.9, n_labels)   # Saturation
    vals = rng.uniform(0.65, 0.95, n_labels) # Value (brightness)
    hsv  = np.stack([hues, sats, vals], axis=1)  # Shape (N, 3)

    rgb  = hsv_to_rgb(hsv)
    return ListedColormap(rgb, name=f'hsv_{n_labels}')

labels   = np.unique(masks)
n_labels = labels.max() + 1
cmap     = large_discrete_cmap(n_labels, seed=42)
norm     = BoundaryNorm(np.arange(-0.5, n_labels + 0.5), n_labels)

def show_overlay(z):
    """Overlay raw image and segmentation mask without a legend."""
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Raw image
    ax[0].imshow(stack[z], cmap='gray')
    ax[0].set_title(f'Raw   z={z}')
    ax[0].axis('off')

    # Mask overlay
    ax[1].imshow(stack[z], cmap='gray')
    ax[1].imshow(masks[z], cmap=cmap, norm=norm, alpha=0.4)
    ax[1].set_title('Mask overlay')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

widgets.interact(
    show_overlay,
    z=widgets.IntSlider(
        min=0,
        max=stack.shape[0]-1,
        step=1,
        value=z_mid,
        description='Z-slice',
        continuous_update=False
    )
);

In [ ]:
# ============================================================
# 5) Save masks as BigTIFF  ──────────────────────────────────
# ============================================================
out_path = './output/' + img_name + '_label.tif'
tif.imwrite(out_path, masks.astype(np.uint16), bigtiff=True)
print("Mask stack written to", out_path)
